# Продвинутое машинное обучение: ДЗ 2
## *Латыпов Ильяс, группа DS-22*
### Часть 1.

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
   - взять в тренировочный набор турниры с dateStart из 2019 года; 
   - в тестовый — турниры с dateStart из 2020 года.

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, SGDRegressor
from scipy.stats import spearmanr, kendalltau
from scipy.special import logit
import zipfile
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

# Загрузка и обработка данных

In [3]:
myzip = zipfile.ZipFile("D:\\ID\\Course\\MADE\\2.Adv_ML\\HW\\chgk.zip")
print("Содержание zip файла:")
for file_name in myzip.namelist():
    print(file_name)

Содержание zip файла:
players.pkl
results.pkl
tournaments.pkl


### Загрузка tournaments.pkl

In [4]:
data_tournament = pd.DataFrame(pickle.load(myzip.open('tournaments.pkl', 'r'))).T
data_tournament["year"] = data_tournament['dateStart'].apply(lambda x: int(x[:4]) if x is not None else None)
data_tournament['sum_questionQty'] = data_tournament['questionQty'].apply(lambda x: sum(x.values()) if x is not None else 0)
train_tournament = data_tournament[data_tournament['year']==2019]
test_tournament = data_tournament[data_tournament['year']==2020]
train_tournament = train_tournament[["id", "year", "sum_questionQty"]]
test_tournament = test_tournament[["id", "year", "sum_questionQty"]]
print("Количество соревнований в 2019 в tournaments.pkl:", train_tournament.shape)
print("Количество соревнований в 2020 в tournaments.pkl:", test_tournament.shape)

Количество соревнований в 2019 в tournaments.pkl: (687, 3)
Количество соревнований в 2020 в tournaments.pkl: (418, 3)


### Загрузка players.pkl

In [4]:
data_players = pd.DataFrame(pickle.load(myzip.open('players.pkl', 'r'))).T
print("Общее количество участников в players.pkl:", data_players.shape)
display(data_players.sample(2))

Общее количество участников в players.pkl: (204063, 4)


,id,name,patronymic,surname
217116,217116,Владимир,Александрович,Варивода
152643,152643,Ульяна,Юрьевна,Кобылец


### Загрузка results.pkl

In [25]:
%%time
results = pickle.load(myzip.open('results.pkl', 'r'))
tournament_test_list = test_tournament["id"].to_list() 
tournament_train_list = train_tournament["id"].to_list()

data_result = pd.DataFrame([])
for tour in (tournament_train_list + tournament_test_list):
    if tour not in results.keys():
        print(f"Не найдено соревнование № {tour} в results.pkl")
        break
    cur_tour_df = pd.DataFrame(results[tour])
    cur_tour_df["tournament"] = tour
    if tour in tournament_train_list:
        cur_tour_df["year"] = 2019
    else:
        cur_tour_df["year"] = 2020
    data_result = pd.concat([data_result, cur_tour_df])
    
data_result["teamMembers_empty"] = data_result["teamMembers"].apply(lambda x: not x) 
data_result = data_result[(~data_result["mask"].isna()) & (~data_result["teamMembers_empty"])]
data_result.drop(columns=["teamMembers_empty"], inplace=True)

data_result["players_id"] = data_result["teamMembers"].apply(lambda x: list(map(lambda y: y['player']['id'], x)))
data_result["players_number"] = data_result["teamMembers"].apply(lambda x: len(list(map(lambda y: y['player']['id'], x))))
data_result["players_name"] = data_result["teamMembers"].apply(lambda x: list(map(lambda y: y['player']['surname'] + " " +
                                                               y['player']['name'] + " " + y['player']['patronymic'], x)))

data_result["team_id"] = data_result["team"].apply(lambda x: x['id'])
data_result = data_result[["tournament", "year", "team_id", "mask", "questionsTotal", 
                           "players_id", "players_name", "players_number"]]

data_result['mask'] = data_result['mask'].apply(lambda x: x.replace("?","0").replace("-","0").replace("X",""))
data_result["mask_length"] = data_result["mask"].apply(len)

temp = pd.DataFrame(data_result.groupby(by = "tournament")["mask_length"].max())
temp.rename(columns = {"mask_length" : "tour_mask_length"}, inplace=True)
data_result = pd.merge(data_result, temp, on='tournament', how="left")
data_result = data_result[data_result["mask_length"] == data_result["tour_mask_length"]]
data_result.drop(columns=["tour_mask_length"], inplace=True)

data_result_ext = data_result.loc[data_result.index.repeat(data_result.players_number)]
data_result_ext = data_result_ext.reset_index()
data_result_ext.drop(columns = ["index"], inplace=True)
data_result_ext = data_result_ext.reset_index()
data_result_ext["list_ind"] = data_result_ext["index"] % data_result_ext["players_number"]
data_result_ext["player_id"] = data_result_ext.apply(lambda x: x["players_id"][x["list_ind"]], axis=1)
data_result_ext["player_name"] = data_result_ext.apply(lambda x: x["players_name"][x["list_ind"]], axis=1)
data_result_ext.drop(columns=["players_id","players_name", "players_number", "list_ind", "index"], inplace=True)
data_result_ext.rename(columns = {"tournament" : "tournament_id"}, inplace=True)

train_result = data_result_ext[data_result_ext["year"] == 2019]
test_result = data_result_ext[data_result_ext["year"] == 2020]
test_result = test_result[test_result["player_id"].isin(set(train_result.player_id.to_list()))]
train_result.to_pickle('result_train.pkl')
test_result.to_pickle('result_test.pkl')

print("Сумма участников во всех турнирах 2019г", train_result.shape)
print("Сумма участников во всех турнирах 2020г:", test_result.shape)
display(train_result.head(2))

Сумма участников во всех турнирах 2019г (414774, 8)
Сумма участников во всех турнирах 2020г: (99757, 8)


,tournament_id,year,team_id,mask,questionsTotal,mask_length,player_id,player_name
0,4772,2019,45556,111111111011111110111111111100010010,28.0,36,6212,Выменец Юрий Яковлевич
1,4772,2019,45556,111111111011111110111111111100010010,28.0,36,18332,Либер Александр Витальевич


Wall time: 1min 40s


In [26]:
print("Выборка соревнований для обучения:", data_result[data_result["year"] == 2019].tournament.nunique())
print("Выборка соревнований для теста:", data_result[data_result["year"] == 2020].tournament.nunique())

Выборка соревнований для обучения: 675
Выборка соревнований для теста: 173


In [8]:
train_result = pd.read_pickle('result_train.pkl')
test_result = pd.read_pickle('result_test.pkl')
train_result.shape, test_result.shape

((414774, 8), (99757, 8))

In [9]:
players_train_ids = set(train_result.player_id.to_list())
players_test_ids = set(test_result.player_id.to_list())
print("Количество участников в 2019г: ", len(players_train_ids))
print("Количество участников в 2020г: ", len(players_test_ids))

Количество участников в 2019г:  57424
Количество участников в 2020г:  22946


### Часть 2.

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
  - повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
  - в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
  - для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Пусть $s_i$ - сила игрока, $q_j$ - сложность вопроса, $\mu$ - среднее, $p(s_i, q_j)$ - вероятность игрока с силой $s_i$ ответить на вопрос с сложностью $q_j$ \
Постpоим модель вида $$p(s_i, q_j) = \sigma(s_i + q_j + \mu)$$ \
Для обучения будем считать, что если комманда ответила на вопрос, то и игроки из этой комманды ответили на этот вопрос.

Далее в коде готовиться повопросный датасет (матрица **onehot векторов** для вопросов и игроков) и обучается логистическая регрессия

In [19]:
%%time
enum_players_ids = {k: i for i, k in enumerate(players_train_ids)}
enum_train_tournament_ids = {k: i for i, k in enumerate(set(train_result["tournament_id"].to_list()))}
enum_test_tournament_ids = {k: i for i, k in enumerate(set(test_result["tournament_id"].to_list()))}
train_result['player_dop_id'] = train_result['player_id'].map(enum_players_ids)
train_result['tournament_dop_id'] = train_result['tournament_id'].map(enum_train_tournament_ids)                                                                   
test_result['player_dop_id'] = test_result['player_id'].map(enum_players_ids)
test_result['tournament_dop_id'] = test_result['tournament_id'].map(enum_test_tournament_ids)

temp = train_result.groupby('tournament_dop_id')['mask_length'].max().cumsum()
question_numbers = temp.iloc[-1]
question_start_ids = dict(temp - temp[0])
X = list()
y = list()
for ind in range(train_result.shape[0]):
    question_start_id = question_start_ids[train_result.iloc[ind]["tournament_dop_id"]]
    player_id = train_result.iloc[ind]['player_dop_id']
    mask = train_result.iloc[ind]['mask']
    for i, question_result in enumerate(mask):
        X.append([question_start_id + i, player_id])                
        if question_result == '0':
            y.append(0)
        else:
            y.append(1)
        
X = np.array(X)
y = np.array(y)

def convert_to_csr_matrix(X, question_numbers):
    x_rows = np.repeat(np.arange(X.shape[0]), 2)
    x_new = X.copy()
    x_new[:,1] += question_numbers #+ 1
    csr_x = csr_matrix((np.ones(X.shape[0] * 2, dtype=np.uint8), (x_rows, x_new.ravel())), dtype=np.uint8)
    return csr_x

X_csr = convert_to_csr_matrix(X, question_numbers)
X_csr.shape, y.shape

Wall time: 2min 45s


((17753544, 90727), (17753544,))

Составлена матрица для обучения состоящия из onehot векторов для вопросов и игроков, проверим совпадение кол-ва столбцов

In [20]:
question_numbers, len(players_train_ids), len(players_train_ids) + question_numbers

(33303, 57424, 90727)

In [56]:
%%time
model = SGDClassifier(loss="log", fit_intercept=True, tol=0.00001, alpha = 0.0000001, n_jobs = -1)
model.fit(X_csr, y)
pickle.dump(model, open('SGDClassifier_LogR.pkl', 'wb'))
print("Mean accuracy:", model.score(X_csr, y))

Mean accuracy: 0.7606019395338756
Wall time: 7min 28s


In [23]:
model = pickle.load(open('SGDClassifier_LogR.pkl', 'rb'))

mu_w = model.intercept_[0]
players_w = model.coef_[0, question_numbers:]
question_w = model.coef_[0, :question_numbers]
print('Среднее (intercept) модели:', mu_w)
print(f'Диапазон весов для описания рейтинга игроков: {players_w.min()} до {players_w.max()}')
print(f'Диапазон весов для описания сложности вопросов: {question_w.min()} до {question_w.max()}')

Среднее (intercept) модели: -1.4627860583493275
Диапазон весов для описания рейтинга игроков: -3.5650055467144 до 3.8179075302495775
Диапазон весов для описания сложности вопросов: -4.976105753534145 до 5.691147393197159


Для получения рейтинга игроков в диапазоне (0, 1) к полученным весам onehot векторов применим функцию сигмоиду

In [24]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

rating = {ind:weight for ind, weight in enumerate(model.coef_[0, question_numbers:])}
ratings = train_result[['player_dop_id', 'player_name']].drop_duplicates()
ratings['rating'] = ratings['player_dop_id'].apply(lambda x: sigmoid(rating[x]))
ratings = ratings.sort_values(by='rating', ascending=False)
ratings = ratings.reset_index(drop=True)
ratings.head(20)

,player_dop_id,player_name,rating
0,9646,Руссо Максим Михайлович,0.978499
1,1452,Брутер Александра Владимировна,0.974202
2,10176,Семушин Иван Николаевич,0.973901
3,9831,Савченков Михаил Владимирович,0.971302
4,10660,Сорожкин Артём Сергеевич,0.969905
5,10700,Спешков Сергей Леонидович,0.969595
6,7120,Мереминский Станислав Григорьевич,0.963511
7,51130,Саксонов Антон Владимирович,0.961107
8,6276,Либер Александр Витальевич,0.961070
9,2368,Гилёв Алексей Владимирович,0.960886


### Часть 3.

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
  - предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
  - в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров

Для расчета метрики будем считать, что если комманда ответила на вопрос то хотя бы один игрок из комманды ответил на вопрос. \
Тогда вероятность комманды ответить на вопрос:

$$p(team_k, q_j) = 1 - \prod_{i = 1}^{N}(1 - p(s_i, q_j)) = 1 - \prod_{i = 1}^{N}(1 - \sigma(s_i + q_j + \mu))$$

В случае если вопрос неизвестен $(q*)$, то в качестве оценки возможно расчитать вероятность ответа комманды на вопрос средней сложности. Но так как на ранжирование комманд сложность вопроса не влияет, то ее и среднее($\mu$) можно занулить
$$p(team_k, q*) = 1 - \prod_{i = 1}^{N}(1 - p(s_i, q*)) = 1 - \prod_{i = 1}^{N}(1 - \sigma(s_i))$$

In [25]:
# Предварительно расчитаем коммандные ранги (вероятности ответа на неизвестный вопрос) на тестовом датасете 
test_result = pd.merge(test_result, ratings[["rating","player_dop_id"]], on='player_dop_id', how="left")
test_result["tournament_team_id"] = test_result.apply(lambda x: str(x["tournament_id"]) + "_" + str(x["team_id"]), axis=1)
train_result = pd.merge(train_result, ratings[["rating","player_dop_id"]], on='player_dop_id', how="left")
train_result["tournament_team_id"] = train_result.apply(lambda x: str(x["tournament_id"]) + "_" + str(x["team_id"]), axis=1)

def team_rating(x):
    return (1 - np.prod(1 - x))

def result_gr(df_result):
    df_result_gr = pd.DataFrame(df_result.groupby(by = "tournament_team_id").questionsTotal.max())
    temp = pd.DataFrame(df_result.groupby(by = "tournament_team_id").rating.agg([team_rating]))
    df_result_gr = pd.merge(df_result_gr, temp, how="left", left_index=True, right_index=True)
    df_result_gr = df_result_gr.reset_index()
    df_result_gr["tournament_id"] = df_result_gr["tournament_team_id"].apply(lambda x: int(x.split("_")[0]))
    df_result_gr["team_id"] = df_result_gr["tournament_team_id"].apply(lambda x: int(x.split("_")[1]))
    return df_result_gr

test_result_gr = result_gr(test_result)
test_result_gr.head(4)

,tournament_team_id,questionsTotal,team_rating,tournament_id,team_id
0,4957_10285,14.0,0.999979,4957,10285
1,4957_1799,12.0,0.999971,4957,1799
2,4957_2,21.0,1.000000,4957,2
3,4957_2421,12.0,0.999955,4957,2421


In [112]:
def spearmanr_calc(x):
    return spearmanr(x["questionsTotal"], x["team_rating"]).correlation

def kendalltau_calc(x):
    return kendalltau(x["questionsTotal"], x["team_rating"]).correlation

spearmanr_score_test = round(test_result_gr.groupby("tournament_id").apply(spearmanr_calc).mean(),5)
kendalltau_score_test = round(test_result_gr.groupby("tournament_id").apply(kendalltau_calc).mean(),5)

print(f"Ранговая корреляция Спирмена на тестовых данных: {spearmanr_score_test}")
print(f"Ранговая корреляция Кендалла на тестовых данных: {kendalltau_score_test}")

Ранговая корреляция Спирмена на тестовых данных: 0.74872
Ранговая корреляция Кендалла на тестовых данных: 0.59325


In [114]:
train_result_gr = result_gr(train_result)

spearmanr_score_train = round(train_result_gr.groupby("tournament_id").apply(spearmanr_calc).mean(),5)
kendalltau_score_train = round(train_result_gr.groupby("tournament_id").apply(kendalltau_calc).mean(),5)    
print(f"Ранговая корреляция Спирмена на тренировочных данных: {spearmanr_score_train}")
print(f"Ранговая корреляция Кендалла на тренировочных данных: {kendalltau_score_train}")

Ранговая корреляция Спирмена на тренировочных данных: 0.78327
Ранговая корреляция Кендалла на тренировочных данных: 0.62835


### Часть 4.

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
  - предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
  - разработайте EM-схему для обучения этой модели, реализуйте её в коде;
  - обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

Введем скрытую переменную $z_{ij}$ - правильный ответ игрока $i$ на вопрос $j$ при условии правильного ответа комманды. 

  - $p(z_{ij}=1|team_{k}=1)$

Основные предположения: 
1. Комманда ответила на вопрос то хотя бы один игрок из комманды ответил на вопрос. Поэтому силу комманды оценим как: \
$p(team_k=1, q_j) = 1 - \prod_{i = 1}^{N}(1 - p(z_{ij}=1))$ \
$p(z_{ij}=1) = \sigma(s_i + q_j + \mu)$
2. Если игрок ответил на вопрос то и комманда ответила на вопрос \
$p(team_k=1, z_{ij}=1) = 1$
2. Команда не ответила на вопрос то все игроки не смогли ответить на него \
$p(z_{ij}=1|team_{k}=0) = 0$
3. Ответы игроков на вопрос независят друг от друга

По теореме Байеса:


$$p(z_{ij}=1|team_{k}=1) = \frac{p(team_{k}=1|z_{ij}=1)p(z_{ij}=1)}{p(team_k=1, q_j)} = 
\frac{\sigma(s_i + q_j + \mu))}{1 -  \prod_{i = 1}^{N}(1 - \sigma(s_i + q_j + \mu))}$$

EM-алгоритм:
- M шаг. Оценка $p(z_{ij})$ с применением логистической регрессии 
 $$p(z_{ij}=1) = \sigma(s_i + q_j + \mu)$$
- E шаг. Оценка $p(z_{ij}=1|team_{k}=1)$

$$p(z_{ij}=1|team_{k}=1) = \frac{\sigma(s_i + q_j + \mu))}{1 -  \prod_{i = 1}^{N}(1 - \sigma(s_i + q_j + \mu))}$$

In [28]:
# Функция для расчета ранговой корреляция Спирмена и Кендалла
def calc_metrics(df_result_train, df_result_test):
    df_result_gr = result_gr(df_result_train)
    spearmanr_score_train = round(df_result_gr.groupby("tournament_id").apply(spearmanr_calc).mean(),5)
    kendalltau_score_train = round(df_result_gr.groupby("tournament_id").apply(kendalltau_calc).mean(),5)
    
    df_result_gr = result_gr(df_result_test)
    spearmanr_score_test = round(df_result_gr.groupby("tournament_id").apply(spearmanr_calc).mean(),5)
    kendalltau_score_test = round(df_result_gr.groupby("tournament_id").apply(kendalltau_calc).mean(),5)  

    print(f"Ранговая корреляция Спирмена. Обучение: {spearmanr_score_train}, тест: {spearmanr_score_test}")
    print(f"Ранговая корреляция Кендалла. Обучение: {kendalltau_score_train}, тест: {kendalltau_score_test}")

In [249]:
# Сбор доп. датасета для EM алгоритма (для переоценки таргета)
temp = train_result.groupby('tournament_dop_id')['mask_length'].max().cumsum()
question_numbers = temp.iloc[-1]
question_start_ids = dict(temp - temp[0])
X_df = list()
y_df = list()
for ind in range(train_result.shape[0]):
    question_start_id = question_start_ids[train_result.iloc[ind]["tournament_dop_id"]]
    player_id = train_result.iloc[ind]['player_dop_id']
    tournament_dop_id = train_result.iloc[ind]["tournament_dop_id"]
    team_id = train_result.iloc[ind]["team_id"]
    
    mask = train_result.iloc[ind]['mask']
    for i, question_result in enumerate(mask):
        X_df.append([question_start_id + i, player_id, tournament_dop_id, team_id])                
        if question_result == '0':
            y_df.append(0)
        else:
            y_df.append(1)
        
y_data = pd.DataFrame(y)
X_data = pd.DataFrame(X_df, columns=["question_id", "player_dop_id", "tournament_dop_id", "team_id"])
X_data["tournament_team_id"] = X_data.apply(lambda x: str(x["tournament_dop_id"]) + "_" + str(x["team_id"]), axis=1)

y_data.to_pickle('y_data.pkl')
X_data.to_pickle('X_data.pkl')

In [89]:
X_data = pd.read_pickle('X_data.pkl')
y_data = pd.read_pickle('y_data.pkl')
print(X_data.shape, y_data.shape)
X_data.head(3)

(17753544, 5) (17753544, 1)


,question_id,player_dop_id,tournament_dop_id,team_id,tournament_team_id
0,1505,2101,16,45556,16_45556
1,1506,2101,16,45556,16_45556
2,1507,2101,16,45556,16_45556


In [118]:
M-шаг: 1 итерация (подготовительная)
eps = 0.0001
model = SGDClassifier(loss="log", fit_intercept=True, tol=0.001, alpha = 0.0000001, n_jobs = -1)
model.fit(X_csr, y)

# расчет метрик
mu_w = model.intercept_[0]
players_w = model.coef_[0, question_numbers:]
question_w = model.coef_[0, :question_numbers]
rating = {ind:weight for ind, weight in enumerate(model.coef_[0, question_numbers:])}
ratings = train_result[['player_dop_id', 'player_name']].drop_duplicates()
ratings['rating'] = ratings['player_dop_id'].apply(lambda x: sigmoid(rating[x]))
ratings = ratings.reset_index(drop=True)
test_result.drop(columns = ["rating"], inplace=True)
train_result.drop(columns = ["rating"], inplace=True)
test_result = pd.merge(test_result, ratings[["rating","player_dop_id"]], on='player_dop_id', how="left")
train_result = pd.merge(train_result, ratings[["rating","player_dop_id"]], on='player_dop_id', how="left")
print("Итерация №1 (подготовительная)")
calc_metrics(train_result, test_result)

# Е-шаг: 1 итерация (подготовительная)
X_data["sigma_ij"] = model.predict_proba(X_csr)[:,1]
temp = pd.DataFrame(X_data.groupby(by = "tournament_team_id")["sigma_ij"].agg([team_rating]))
temp = temp.reset_index()
if "team_rating" in X_data.columns:
    X_data.drop(columns=["team_rating"],inplace=True)
X_data = pd.merge(X_data, temp, how="left", on="tournament_team_id")
X_data["y_new_iter"] = y * X_data["sigma_ij"].values / X_data["team_rating"].values
y_new = np.clip(X_data["y_new_iter"].values, eps, 1 - eps)

Итерация №1 (подготовительная)
Ранговая корреляция Спирмена. Обучение: 0.78327, тест: 0.74872
Ранговая корреляция Кендалла. Обучение: 0.62835, тест: 0.59325


In [122]:
iter_number = 4
for k in range(1, iter_number):
    # M-шаг
    model = SGDRegressor(fit_intercept=True, tol=0.001, alpha = 0.0000001)
    model.fit(X_csr, logit(y_new))

    # расчет метрик
    mu_w = model.intercept_[0]
    players_w = model.coef_[question_numbers:]
    question_w = model.coef_[:question_numbers]
    rating = {ind:weight for ind, weight in enumerate(model.coef_[question_numbers:])}
    ratings = train_result[['player_dop_id', 'player_name']].drop_duplicates()
    ratings['rating'] = ratings['player_dop_id'].apply(lambda x: sigmoid(rating[x]))
    ratings = ratings.reset_index(drop=True)
    test_result.drop(columns = ["rating"], inplace=True)
    train_result.drop(columns = ["rating"], inplace=True)
    test_result = pd.merge(test_result, ratings[["rating","player_dop_id"]], on='player_dop_id', how="left")
    train_result = pd.merge(train_result, ratings[["rating","player_dop_id"]], on='player_dop_id', how="left")
    print(f"Итерация №{k+1}")
    calc_metrics(train_result, test_result)

    # Е-шаг
    X_data["sigma_ij"] = sigmoid(model.predict(X_csr))
    temp = pd.DataFrame(X_data.groupby(by = "tournament_team_id")["sigma_ij"].agg([team_rating]))
    temp = temp.reset_index()
    if "team_rating" in X_data.columns:
        X_data.drop(columns=["team_rating"],inplace=True)
    X_data = pd.merge(X_data, temp, how="left", on="tournament_team_id")
    X_data["y_new_iter"] = y * X_data["sigma_ij"].values / X_data["team_rating"].values
    y_new = X_data["y_new_iter"].values
    y_new = np.clip(X_data["y_new_iter"].values, eps, 1 - eps)

Итерация №2
Ранговая корреляция Спирмена. Обучение: 0.80141, тест: 0.75217
Ранговая корреляция Кендалла. Обучение: 0.63482, тест: 0.59826
Итерация №3
Ранговая корреляция Спирмена. Обучение: 0.81236, тест: 0.76428
Ранговая корреляция Кендалла. Обучение: 0.65976, тест: 0.60628
Итерация №4
Ранговая корреляция Спирмена. Обучение: 0.81691, тест: 0.77284
Ранговая корреляция Кендалла. Обучение: 0.66347, тест: 0.61551


In [125]:
ratings = ratings.sort_values(by='rating', ascending=False)
ratings.head(10)

,player_dop_id,player_name,rating
8223,9646,Руссо Максим Михайлович,0.998366
5957,1452,Брутер Александра Владимировна,0.998150
1208,10176,Семушин Иван Николаевич,0.997753
1212,9831,Савченков Михаил Владимирович,0.997069
155,10660,Сорожкин Артём Сергеевич,0.996783
8244,10700,Спешков Сергей Леонидович,0.996272
1207,2368,Гилёв Алексей Владимирович,0.994178
1209,7120,Мереминский Станислав Григорьевич,0.993720
9664,51130,Саксонов Антон Владимирович,0.992919
9656,4105,Иванцова Светлана Сергеевна,0.992611


### Часть 5.

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

Cложность турнира будем расичитвать как среднее сложности вопросов. Сложность вопросов расчитаем как функция сигмоида от весов, для того, что сложность вопросов определить в диапазоне (0,1) 

In [236]:
question_w = sigmoid(-model.coef_[:question_numbers])
q_rating = train_result[['tournament_id', 'mask_length', 'tournament_dop_id']].drop_duplicates()
q_rating = pd.merge(q_rating, data_tournament[["id","name"]].rename(columns={"id":"tournament_id"}), 
                    on='tournament_id', how="left")

q_rating["q_rating"] = 0
for ind in range(q_rating.shape[0]):
    tournament_dop_id = q_rating.iloc[ind]['tournament_dop_id']
    mask_length = q_rating.iloc[ind]['mask_length']
    start_ind = question_start_ids[tournament_dop_id]
    q_rating["q_rating"].iloc[ind] = question_w[start_ind:start_ind+mask_length].mean()

In [237]:
print('Рейтинг-лист сложных турниров:')
display(q_rating[["q_rating", "name"]].sort_values(by = ['q_rating'], ascending=False).head(10))

Рейтинг-лист сложных турниров:


,q_rating,name
663,0.925777,Чемпионат Санкт-Петербурга. Первая лига
541,0.891286,Угрюмый Ёрш
369,0.864154,Синхрон высшей лиги Москвы
43,0.848152,Первенство правого полушария
284,0.815950,Записки охотника
130,0.798486,Серия Premier. Седьмая печать
13,0.796630,Кубок городов
639,0.793592,Воображаемый музей
430,0.788196,Синхрон Кеста
404,0.787111,All Cats Are Beautiful


In [238]:
print('Рейтинг-лист простых турниров:')
display(q_rating[["q_rating", "name"]].sort_values(by = ['q_rating'], ascending=True).head(10))

Рейтинг-лист простых турниров:


,q_rating,name
154,0.217483,Синхрон Лиги Разума
399,0.220844,Синхрон-lite. Выпуск XXX
7,0.233805,(а)Синхрон-lite. Лига старта. Эпизод III
11,0.246065,(а)Синхрон-lite. Лига старта. Эпизод V
38,0.246713,Лига Сибири. VI тур.
598,0.248755,Второй тематический турнир имени Джоуи Триббиани
221,0.272902,KFC
36,0.280000,Лига Сибири. IV тур.
483,0.288196,Синхрон простых вопросов. Зима
529,0.291307,Осенняя кинолига


### Выводы

  - Baseline-модель демонстрирует хорошие метрики качества, результат хоть и немного хуже, но близок к результату EM алгоритма. 
  - В рейтинге сильных игроков присутствуют известные сильные игроки, что повышает доверие к моделям
  - EM-алгоритм обучается на порядок медленнее. В ходе итераций EM алгоритма метрика качества росла. 
  - EM-алгоритм при различных запусках может ухудшать качество модели на последующих итерациях, поэтому требуется каждую модель и результаты фиксировать, что бы можно было провести в дальнейшем селекцию при необходимости
  - Сложность вопросов так же выучена достаточно хорошо, что косвенно подтверждается оценкой сложности турниров